# MPI Programming in C

This notebook is a introduction to MPI programming in C. It covers the following topics:

1. Basic MPI Environment: Initialization, Rank, Size, Finalization
2. Blocking Communication: Blocking Send/Recv
3. Non-blocking Communication: MPI_Isend, MPI_Irecv, MPI_Wait, MPI_Waitall
4. Collective Communication: Broadcast, Scatter, Gather, Reduce, Allreduce, All-to-All
5. Derived Datatypes: Creating custom MPI datatypes for complex structures
6. Subcommunicators: Splitting MPI_COMM_WORLD into groups
7. Topology Mapping: Cartesian grid communicators
8. Performance Measurement: MPI_Wtime for timing

Compile with `mpicc` and run with `mpiexec` or `mpirun`.

For example:
```bash
mpiexec -n 4 ./mpi_example
```

## 1. Basic MPI Environment

Initialize the MPI environment, get each process's rank and the total number of processes, then finalize.

`MPI_Init` : Initialize the MPI environment

`MPI_Comm_rank` : Get the rank of the calling process in the communicator

`MPI_Comm_size` : Get the total number of processes in the communicator

`MPI_Finalize` : Finalize the MPI environment

Code example

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    printf("[Basic] Process %d of %d\n", rank, size);
    MPI_Finalize();
    return 0;
}
```


In [63]:
%%bash
# Compile and run
mpicc ./codes/1_hello_basic.c -o codes/1_hello_basic
mpiexec -n 4 ./codes/1_hello_basic

[Basic] Process 0 of 4
[Basic] Process 1 of 4
[Basic] Process 2 of 4
[Basic] Process 3 of 4


## 2. Blocking Communication

Blocking communication using `MPI_Send` and `MPI_Recv`. Process 0 sends a message to process 1.



`MPI_Send` : Send a message to another process

`MPI_Recv` : Receive a message from another process

Code example

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char *argv[]) {
  MPI_Init(&argc, &argv);
  int rank, size;
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);
  if (size < 2) {
    if (rank == 0)
      fprintf(stderr, "Requires at least 2 processes\n");
    MPI_Finalize();
    return 1;
  }
  if (rank == 0) {
    char msg[] = "Hello from 0";
    MPI_Send(msg, sizeof(msg), MPI_CHAR, 1, 0, MPI_COMM_WORLD);
    printf("[Blocking] Process 0 sent message.\n");
  } else if (rank == 1) {
    char msg[50];
    MPI_Recv(msg, 50, MPI_CHAR, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    printf("[Blocking] Process 1 received: %s\n", msg);
  }
  MPI_Finalize();
  return 0;
}
```




In [65]:
%%bash
# Compile and run
mpicc ./codes/2_blocking_comm.c -o ./codes/2_blocking_comm
mpiexec -n 2 ./codes/2_blocking_comm

[Blocking] Process 0 sent message.
[Blocking] Process 1 received: Hello from 0


## 3. Non-blocking Communication

Using `MPI_Isend` and `MPI_Irecv` to allow computation and communication to overlap. Process 0 sends a message to process 1 and continues computation while process 1 receives the message in the background with a sleep to simulate computation.

`MPI_Request` : Handle to a non-blocking operation

`MPI_Wait` : used to wait for non-blocking operations to complete.

`MPI_Isend` : Non-blocking send

`MPI_Irecv` : Non-blocking receive

Code example

```c
#include <mpi.h>
#include <stdio.h>
#include <unistd.h>

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    if (size < 2) {
        if (rank == 0) fprintf(stderr, "Requires at least 2 processes\n");
        MPI_Finalize();
        return 1;
    }
    MPI_Request req;
    if (rank == 0) {
        int data = 123;
        MPI_Isend(&data, 1, MPI_INT, 1, 1, MPI_COMM_WORLD, &req);
        printf("[Nonblocking] Process 0 initiated Isend with data %d\n", data);
        sleep(1);
        MPI_Wait(&req, MPI_STATUS_IGNORE);
        printf("[Nonblocking] Process 0 completed Isend\n");
    } else if (rank == 1) {
        int data = 0;
        MPI_Irecv(&data, 1, MPI_INT, 0, 1, MPI_COMM_WORLD, &req);
        printf("[Nonblocking] Process 1 initiated Irecv\n");
        sleep(1);
        MPI_Wait(&req, MPI_STATUS_IGNORE);
        printf("[Nonblocking] Process 1 received data %d\n", data);
    }
    MPI_Finalize();
    return 0;
}
```


In [66]:
%%bash
# Compile and run
mpicc ./codes/3_nonblocking.c -o ./codes/3_nonblocking
mpiexec -n 2 ./codes/3_nonblocking

[Nonblocking] Process 1 initiated Irecv
[Nonblocking] Process 0 initiated Isend with data 123
[Nonblocking] Process 0 completed Isend
[Nonblocking] Process 1 received data 123


## 4. Collective Communication

Demonstrate broadcast, scatter, gather, reduce, allreduce, and all-to-all collective operations.

`MPI_Bcast` : Broadcast a message from one process to all others

`MPI_Scatter` : Scatter an array from one process to all others

`MPI_Gather` : Gather an array from all processes to one

`MPI_Reduce` : Reduce an array from all processes to one

`MPI_Allreduce` : Reduce an array from all processes to all

`MPI_Alltoall` : Exchange data between all processes

Code example

```c
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

int main(int argc, char **argv) {
  int rank, size;

  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  // MPI_Bcast: Broadcast a value from rank 0 to all processes.
  int bcast_data;
  if (rank == 0)
    bcast_data = 100;
  MPI_Bcast(&bcast_data, 1, MPI_INT, 0, MPI_COMM_WORLD);
  printf("[Collective] Rank %d received bcast_data: %d\n", rank, bcast_data);
  sleep(1);

  // MPI_Scatter: Scatter an array of integers from rank 0.
  int *scatter_sendbuf = NULL;
  int scatter_recvbuf;
  if (rank == 0) {
    scatter_sendbuf = malloc(sizeof(int) * size);
    for (int i = 0; i < size; i++) {
      scatter_sendbuf[i] = (i + 1) * 10;
    }
  }
  MPI_Scatter(scatter_sendbuf, 1, MPI_INT, &scatter_recvbuf, 1, MPI_INT, 0,
              MPI_COMM_WORLD);
  printf("[Collective] Rank %d received scatter_recvbuf: %d\n", rank,
         scatter_recvbuf);
  sleep(1);

  // MPI_Gather: Gather a single integer from each process to rank 0.
  int send_val = rank;
  int *gather_recvbuf = NULL;
  if (rank == 0) {
    gather_recvbuf = malloc(sizeof(int) * size);
  }
  MPI_Gather(&send_val, 1, MPI_INT, gather_recvbuf, 1, MPI_INT, 0,
             MPI_COMM_WORLD);
  if (rank == 0) {
    printf("[Collective] Rank 0 gathered values: ");
    for (int i = 0; i < size; i++) {
      printf("%d ", gather_recvbuf[i]);
    }
    printf("\n");
  }
  sleep(1);

  // MPI_Reduce: Sum the values from each process, result goes to rank 0.
  int reduce_sum;
  MPI_Reduce(&send_val, &reduce_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
  if (rank == 0) {
    printf("[Collective] Sum of ranks (reduce): %d\n", reduce_sum);
  }
  sleep(1);

  // MPI_Allreduce: Sum the values from each process. All processes receive the
  // result.
  int allreduce_sum;
  MPI_Allreduce(&send_val, &allreduce_sum, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);
  printf("[Collective] Rank %d has allreduce sum: %d\n", rank, allreduce_sum);
  sleep(1);

  // MPI_Alltoall: Exchange data between all processes.
  int *alltoall_send = malloc(sizeof(int) * size);
  int *alltoall_recv = malloc(sizeof(int) * size);
  for (int i = 0; i < size; i++) {
    alltoall_send[i] = rank * 100 + i;
  }
  MPI_Alltoall(alltoall_send, 1, MPI_INT, alltoall_recv, 1, MPI_INT,
               MPI_COMM_WORLD);
  printf("[Collective] Rank %d received alltoall data: ", rank);
  for (int i = 0; i < size; i++) {
    printf("%d ", alltoall_recv[i]);
  }
  printf("\n");

  // Cleanup
  free(alltoall_send);
  free(alltoall_recv);
  if (rank == 0) {
    free(scatter_sendbuf);
    free(gather_recvbuf);
  }

  MPI_Finalize();
  return 0;
}
```


In [67]:
%%bash

mpicc ./codes/4_collective.c -o ./codes/4_collective
mpiexec -n 4 ./codes/4_collective


[Collective] Rank 0 received bcast_data: 100
[Collective] Rank 1 received bcast_data: 100
[Collective] Rank 2 received bcast_data: 100
[Collective] Rank 3 received bcast_data: 100
[Collective] Rank 0 received scatter_recvbuf: 10
[Collective] Rank 1 received scatter_recvbuf: 20
[Collective] Rank 2 received scatter_recvbuf: 30
[Collective] Rank 3 received scatter_recvbuf: 40
[Collective] Rank 0 gathered values: 0 1 2 3 
[Collective] Sum of ranks (reduce): 6
[Collective] Rank 0 has allreduce sum: 6
[Collective] Rank 1 has allreduce sum: 6
[Collective] Rank 2 has allreduce sum: 6
[Collective] Rank 3 has allreduce sum: 6
[Collective] Rank 0 received alltoall data: 0 100 200 300 
[Collective] Rank 1 received alltoall data: 1 101 201 301 
[Collective] Rank 2 received alltoall data: 2 102 202 302 
[Collective] Rank 3 received alltoall data: 3 103 203 303 


## 5. Derived Datatypes

Define a custom MPI datatype for a complex structure. This allows sending a structure with multiple fields in one call.

`MPI_Type_create_struct` : Create a custom MPI datatype from a struct

`MPI_Type_commit` : Commit the custom datatype

`MPI_Type_free` : Free the custom datatype

`MPI_Aint` : Address type for MPI

Code example

```c
#include <mpi.h>
#include <stdio.h>
#include <stddef.h>

typedef struct {
    int id;
    double value;
    int data[3];
} Record;

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    if (size < 2) {
        if (rank == 0) fprintf(stderr, "Requires at least 2 processes\n");
        MPI_Finalize();
        return 1;
    }

    MPI_Datatype record_type;
    int blocklengths[3] = {1, 1, 3};
    MPI_Aint offsets[3];
    offsets[0] = offsetof(Record, id);
    offsets[1] = offsetof(Record, value);
    offsets[2] = offsetof(Record, data);
    MPI_Datatype types[3] = {MPI_INT, MPI_DOUBLE, MPI_INT};
    MPI_Type_create_struct(3, blocklengths, offsets, types, &record_type);
    MPI_Type_commit(&record_type);

    if (rank == 0) {
        Record rec = {42, 3.14159, {7,8,9}};
        MPI_Send(&rec, 1, record_type, 1, 0, MPI_COMM_WORLD);
        printf("[Derived] Process 0 sent record: id=%d, value=%f\n", rec.id, rec.value);
    } else if (rank == 1) {
        Record rec;
        MPI_Recv(&rec, 1, record_type, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("[Derived] Process 1 received record: id=%d, value=%f\n", rec.id, rec.value);
    }
    MPI_Type_free(&record_type);
    MPI_Finalize();
    return 0;
}
```

In [68]:
%%bash
# Compile and run
mpicc ./codes/5_derived_datatype.c -o ./codes/5_derived_datatype
mpiexec -n 2 ./codes/5_derived_datatype

[Derived] Process 0 sent record: id=42, value=3.141590
[Derived] Process 1 received record: id=42, value=3.141590


## 6. Subcommunicators

Split `MPI_COMM_WORLD` into subcommunicators. In this example, processes are divided based on even/odd ranks.

`MPI_Comm_split` : Split a communicator into subgroups

`MPI_Comm_free` : Free a communicator

Code example

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    int color = rank % 2;
    MPI_Comm subcomm;
    MPI_Comm_split(MPI_COMM_WORLD, color, rank, &subcomm);
    int subrank, subsize;
    MPI_Comm_rank(subcomm, &subrank);
    MPI_Comm_size(subcomm, &subsize);
    printf("[Subcomm] Global rank %d in group %d, subrank %d (subsize %d)\n", rank, color, subrank, subsize);
    MPI_Comm_free(&subcomm);
    MPI_Finalize();
    return 0;
}
```

In [69]:
%%bash
# Compile and run
mpicc ./codes/6_subcommunicators.c -o ./codes/6_subcommunicators
mpiexec -n 4 ./codes/6_subcommunicators


[Subcomm] Global rank 0 in group 0, subrank 0 (subsize 2)
[Subcomm] Global rank 2 in group 0, subrank 1 (subsize 2)
[Subcomm] Global rank 1 in group 1, subrank 0 (subsize 2)
[Subcomm] Global rank 3 in group 1, subrank 1 (subsize 2)


## 7. Topology Mapping: Cartesian Grids

Create a Cartesian communicator for applications with grid structure.

`MPI_Dims_create` : Create dimensions for a Cartesian grid

`MPI_Cart_create` : Create a Cartesian communicator

`MPI_Cart_coords` : Get the coordinates of a process in a Cartesian grid

Code example

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    int dims[2] = {0, 0};
    MPI_Dims_create(size, 2, dims);
    int periods[2] = {1, 1};
    MPI_Comm cart_comm;
    MPI_Cart_create(MPI_COMM_WORLD, 2, dims, periods, 1, &cart_comm);
    int cart_rank, coords[2];
    MPI_Comm_rank(cart_comm, &cart_rank);
    MPI_Cart_coords(cart_comm, cart_rank, 2, coords);
    printf("[Cartesian] Global rank %d has coords: (%d, %d)\n", rank, coords[0], coords[1]);
    MPI_Comm_free(&cart_comm);
    MPI_Finalize();
    return 0;
}
```

In [70]:
%%bash
# Compile and run
mpicc ./codes/7_cartesian_topology.c -o ./codes/7_cartesian_topology
mpiexec -n 4 ./codes/7_cartesian_topology


[Cartesian] Global rank 0 has coords: (0, 0)
[Cartesian] Global rank 1 has coords: (0, 1)
[Cartesian] Global rank 2 has coords: (1, 0)
[Cartesian] Global rank 3 has coords: (1, 1)


## 8. Performance Measurement

Use `MPI_Wtime()` to measure the time taken by sections of code. This is critical for tuning parallel applications.

`MPI_Barrier` : Synchronize all processes

`MPI_Wtime` : Get the current time in seconds

Code example

```c
#include <mpi.h>
#include <stdio.h>
#include <unistd.h>

int main(int argc, char *argv[]) {
  MPI_Init(&argc, &argv);
  MPI_Barrier(MPI_COMM_WORLD);
  double start = MPI_Wtime();
  MPI_Barrier(MPI_COMM_WORLD);
  sleep(1);
  double end = MPI_Wtime();
  int rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  if (rank == 0) {
    printf("[Performance] Barrier took %f seconds\n", end - start);
  }
  MPI_Finalize();
  return 0;
}
```

In [71]:
%%bash
# Compile and run
mpicc ./codes/8_performance_measure.c -o ./codes/8_performance_measure
mpiexec -n 4 ./codes/8_performance_measure


[Performance] Barrier took 1.005063 seconds


## 9. Simple example for calculating PI using MPI



```c
#include <math.h>
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[]) {
  int n;                      // total number of intervals
  int i;                      // loop variable
  int rank, size;             // MPI process rank and number of processes
  double h;                   // width of each interval
  double x;                   // sample point in the interval
  double local_sum = 0.0, pi; // partial sum and final result

  // Initialize MPI
  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  // Process 0 reads the command-line argument.
  if (rank == 0) {
    if (argc != 2) {
      fprintf(stderr, "Usage: %s <num_intervals>\n", argv[0]);
      MPI_Abort(MPI_COMM_WORLD, 1);
    }
    n = atoll(argv[1]);
  }
  double t_start = MPI_Wtime();

  // Broadcast the number of intervals to all processes
  MPI_Bcast(&n, 1, MPI_INT, 0, MPI_COMM_WORLD);

  // Compute the width of each interval
  h = 1.0 / n;

  /*
   * Each process computes its portion of the sum.
   * Process 'rank' handles the intervals:
   *   i = rank, rank + size, rank + 2*size, ...
   *
   * The approximation to the integral for each interval is:
   *   (1/n) * (4 / (1 + x*x))   where x = i/n.
   */
  for (i = rank; i < n; i += size) {
    x = i * h;
    local_sum += 4.0 / (1.0 + x * x);
  }
  local_sum = local_sum * h;

  // Use MPI_Reduce to sum up all the partial results into pi (only on rank 0)
  MPI_Reduce(&local_sum, &pi, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

  double t_end = MPI_Wtime();
  if (rank == 0) {
    printf("Calculated Pi = %.16f\n", pi);
    printf("Computation Time: %.10f seconds\n", t_end - t_start);
    double expected_pi = 3.141592653589793;
    double error = fabs(pi - expected_pi);
    printf("Error: %.8e\n", error);
  }

  MPI_Finalize();
  return 0;
}

```

In [1]:
%%bash
# Compile and run
# Usage ./codes/9_final <number of elements>

mpicc ./codes/9_final.c -o ./codes/9_final
mpiexec -n 4 ./codes/9_final 1000000000

Calculated Pi = 3.1415926545897164
Computation Time: 0.3780540000 seconds
Error: 9.99923255e-10
